# Context Engine

Copyright 2025, Denis Rothman


**Building the Context Engine**

*From a Team of Agents to an Intelligent System*


Trong các chương trước, chúng ta đã xây dựng (**engineered**) các **contexts** riêng lẻ và các **specialist agents**. Tuy nhiên, khi hệ thống phát triển, việc quản lý các **agents** này theo một trình tự tuyến tính (**linear sequence**) cố định sẽ trở nên khó khăn và cứng nhắc. Bước tiến hóa tiếp theo là tạo ra một hệ thống có khả năng tư duy, lập kế hoạch và điều phối (**orchestrate**) các **agents** này một cách linh hoạt (**dynamically**) để đạt được mục tiêu cấp cao (**high-level goal**).

Notebook này sẽ giới thiệu về **Context Engine**, một bộ điều khiển thông minh (**intelligent controller**) được thiết kế để chuyển đổi một yêu cầu mơ hồ từ người dùng thành một đầu ra được tạo ra cẩn thận và có khả năng nhận biết ngữ cảnh (**context-aware**). Nó đóng vai trò là một **orchestrator**, thực hiện ủy thác trách nhiệm cho các thành phần chuyên biệt thay vì tự mình giải quyết toàn bộ tác vụ.

### Đột phá then chốt: Lập kế hoạch linh hoạt dựa trên LLM (Dynamic, LLM-Powered Planning)

Cải tiến thực sự trong chương này là việc loại bỏ các **hardcoded workflows** (luồng công việc được lập trình cứng). Chúng ta sẽ xây dựng một **Planner** sử dụng một **LLM** bên ngoài để phân tích mục tiêu của người dùng. Bằng cách tham chiếu một **registry** của các công cụ (**tools**) hiện có, **Planner** này sẽ tạo ra một bản kế hoạch **JSON** đa bước tùy chỉnh một cách tức thời (**on the fly**). Thiết kế mạnh mẽ này giúp tách biệt giữa "việc cần làm" (mục tiêu) và "cách thực hiện" (kế hoạch).

---

### Các thành phần bạn sẽ xây dựng trong Notebook này:

* **The Specialist Agents:** Bao gồm `Librarian`, `Researcher`, và `Writer` từ các bài học trước, chịu trách nhiệm xử lý phong cách (**style**), dữ kiện (**facts**) và tạo nội dung (**content generation**).
* **The Agent Registry:** Một "bộ công cụ" (**toolkit**) mô tả khả năng của từng **agent**, giúp chúng có thể được tìm thấy và sử dụng bởi **Planner**.
* **The Engine's "Brain"**: Bộ điều phối cốt lõi, bao gồm:
* **Planner**: Tạo ra kế hoạch chiến lược.
* **Executor**: Thực thi kế hoạch và quản lý **Context Chaining**, nơi đầu ra của một **agent** sẽ trở thành đầu vào của **agent** tiếp theo một cách liền mạch.
* **Tracer**: Ghi lại toàn bộ quá trình (**logging**) để phục vụ mục đích minh bạch và gỡ lỗi (**debugging**).

![](../images/image_04.png)

In [2]:
# Imports and API Key Setup
# We will use the OpenAI library to interact with the LLM and Google Colab's
# secret manager to securely access your API key.

import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

# The client will automatically read the OPENAI_API_KEY from your environment.
client = OpenAI()
print("OpenAI client initialized.")

OpenAI client initialized.


In [3]:
# Configuration
EMBEDDING_MODEL = "text-embedding-3-small"
EMBEDDING_DIM = 1536 # Dimension for text-embedding-3-small
GENERATION_MODEL = "gpt-5.1"

In [4]:
# Imports for this notebook
import json
import time
from tqdm.auto import tqdm
import tiktoken
from pinecone import Pinecone, ServerlessSpec
from tenacity import retry, stop_after_attempt, wait_random_exponential
# general imports required in the notebooks of this book
import re
import textwrap
from IPython.display import display, Markdown
import copy

/home/dino/Documents/learning-2026/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# 2.Initialize Clients
# --- Initialize Clients (assuming this is already done) ---

# --- Initialize Pinecone Client ---
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
pc = Pinecone(api_key=PINECONE_API_KEY)

# --- Define Index and Namespaces (assuming this is already done) ---
INDEX_NAME = 'genai-mas-mcp-ch3'
NAMESPACE_KNOWLEDGE = "KnowledgeStore"
NAMESPACE_CONTEXT = "ContextLibrary"
spec = ServerlessSpec(cloud='aws', region='us-east-1')

# Check if index exists
if INDEX_NAME not in pc.list_indexes().names():
    print(f"Index '{INDEX_NAME}' not found. Creating new serverless index...")
    pc.create_index(
        name=INDEX_NAME,
        dimension=EMBEDDING_DIM, # Make sure EMBEDDING_DIM is defined
        metric='cosine',
        spec=spec
    )
    # Wait for index to be ready
    while not pc.describe_index(INDEX_NAME).status['ready']:
        print("Waiting for index to be ready...")
        time.sleep(1)
    print("Index created successfully. It is new and empty.")
else:
    # This block runs ONLY if the index already existed.
    print(f"Index '{INDEX_NAME}' already exists.")
    print("Clearing namespaces for a fresh start...")

    # Connect to the index to perform operations
    index = pc.Index(INDEX_NAME)

Index 'genai-mas-mcp-ch3' already exists.
Clearing namespaces for a fresh start...


# 3.Helper Functions (LLM, Embeddings, and MCP)

In [9]:
#3. Helper Functions (LLM, Embeddings, MCP, Pinecone)
# -------------------------------------------------------------------------
# Utility functions to standardize interactions.
# -------------------------------------------------------------------------

# === LLM Interaction ===
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def call_llm_robust(system_prompt, user_prompt,json_mode=False):
    """A centralized function to handle all LLM interactions with retries."""
    try:
        response_format = {"type": "json_object"} if json_mode else {"type": "text"}
        response = client.chat.completions.create(
            model=GENERATION_MODEL,
            response_format=response_format,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error calling LLM: {e}")
        # Raise the exception so the caller can handle it or the engine can stop
        raise e

# === Embeddings ===
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def get_embedding(text):
    """Generates embeddings for a single text query with retries."""
    text = text.replace("\n", " ")
    response = client.embeddings.create(input=[text], model=EMBEDDING_MODEL)
    return response.data[0].embedding

# === Model Context Protocol (MCP) ===
def create_mcp_message(sender, content, metadata=None):
    """Creates a standardized MCP message."""
    return {
        "protocol_version": "2.0 (Context Engine)",
        "sender": sender,
        "content": content, # The actual payload/context
        "metadata": metadata or {}
    }

# === Pinecone Interaction ===
def query_pinecone(query_text, namespace, top_k=1):
    """Embeds the query text and searches the specified Pinecone namespace."""
    try:
        query_embedding = get_embedding(query_text)
        response = index.query(
            vector=query_embedding,
            namespace=namespace,
            top_k=top_k,
            include_metadata=True
        )
        return response['matches']
    except Exception as e:
        print(f"Error querying Pinecone (Namespace: {namespace}): {e}")
        raise e

print("Helper functions defined.")

Helper functions defined.


In [10]:
#@title 4.The Specialist Agents (The Handlers)
# -------------------------------------------------------------------------
# We define the specialist agents. These are largely reused from Chapter 3,
# but enhanced to handle more flexible inputs required for dynamic planning.
# Agents return the raw data (string) as the MCP 'content' for simplicity.
# -------------------------------------------------------------------------

# === 4.1. Context Librarian Agent (Procedural RAG) ===
def agent_context_librarian(mcp_message):
    """
    Retrieves the appropriate Semantic Blueprint from the Context Library.
    """
    print("\n[Librarian] Activated. Analyzing intent...")
    # Extract the specific input required by this agent
    requested_intent = mcp_message['content'].get('intent_query')

    if not requested_intent:
        raise ValueError("Librarian requires 'intent_query' in the input content.")

    # Query Pinecone Context Namespace
    results = query_pinecone(requested_intent, NAMESPACE_CONTEXT, top_k=1)

    if results:
        match = results[0]
        print(f"[Librarian] Found blueprint '{match['id']}' (Score: {match['score']:.2f})")
        # Retrieve the blueprint JSON string stored in metadata
        blueprint_json = match['metadata']['blueprint_json']
        # The output content IS the blueprint itself (as a string)
        content = blueprint_json
    else:
        print("[Librarian] No specific blueprint found. Returning default.")
        # Fallback default
        content = json.dumps({"instruction": "Generate the content neutrally."})

    return create_mcp_message("Librarian", content)

# === 4.2. Researcher Agent (Factual RAG) ===
def agent_researcher(mcp_message):
    """
    Retrieves and synthesizes factual information from the Knowledge Base.
    """
    print("\n[Researcher] Activated. Investigating topic...")
    # Extract the specific input required by this agent
    topic = mcp_message['content'].get('topic_query')

    if not topic:
        raise ValueError("Researcher requires 'topic_query' in the input content.")

    # Query Pinecone Knowledge Namespace
    results = query_pinecone(topic, NAMESPACE_KNOWLEDGE, top_k=3)

    if not results:
        print("[Researcher] No relevant information found.")
        # Return a string indicating no data found
        return create_mcp_message("Researcher", "No data found on the topic.")

    # Synthesize the findings (Retrieve-and-Synthesize)
    print(f"[Researcher] Found {len(results)} relevant chunks. Synthesizing...")
    source_texts = [match['metadata']['text'] for match in results]

    system_prompt = """Bạn là một chuyên gia AI về tổng hợp nghiên cứu (research synthesis). 
Hãy tổng hợp các văn bản nguồn (source texts) đã cung cấp thành một bản tóm tắt ngắn gọn, trình bày dưới dạng danh sách (bullet-pointed summary) và bám sát chủ đề của người dùng. 
Yêu cầu:
1. Tập trung nghiêm ngặt vào các sự kiện thực tế (facts) có trong nguồn tài liệu. 
2. Tuyệt đối không thêm thông tin bên ngoài (outside information)."""
    user_prompt = f"Topic: {topic}\n\nSources:\n" + "\n\n---\n\n".join(source_texts)

    # Use a low temperature for factual synthesis
    findings = call_llm_robust(system_prompt, user_prompt)

    # The output content IS the findings (as a string)
    return create_mcp_message("Researcher", findings)

# === 4.3. Writer Agent (Generation) ===
def agent_writer(mcp_message):
    """
    Combines the factual research with the semantic blueprint to generate the final output.
    Crucially enhanced to handle either raw facts OR previous content for rewriting tasks.
    """
    print("\n[Writer] Activated. Applying blueprint to source material...")

    # Extract inputs.
    blueprint_json_string = mcp_message['content'].get('blueprint')
    # Check for 'facts' first, then 'previous_content'
    facts = mcp_message['content'].get('facts')
    previous_content = mcp_message['content'].get('previous_content')

    if not blueprint_json_string:
        raise ValueError("Writer requires 'blueprint' in the input content.")

    # Determine the source material and label for the prompt
    if facts:
        source_material = facts
        source_label = "RESEARCH FINDINGS"
    elif previous_content:
        source_material = previous_content
        source_label = "PREVIOUS CONTENT (For Rewriting)"
    else:
        raise ValueError("Writer requires either 'facts' or 'previous_content'.")


    # The Writer's System Prompt incorporates the dynamically retrieved blueprint
    system_prompt = f"""Bạn là một chuyên gia AI về tạo nội dung (content generation).
    Nhiệm vụ của bạn là tạo nội dung dựa trên các KẾT QUẢ NGHIÊN CỨU (RESEARCH FINDINGS) được cung cấp.
    Quan trọng nhất, bạn PHẢI cấu trúc, định hình phong cách và thiết lập các giới hạn cho kết quả đầu ra dựa trên các quy tắc được định nghĩa trong BẢN THIẾT KẾ NGỮ NGHĨA (SEMANTIC BLUEPRINT) dưới đây.

    --- SEMANTIC BLUEPRINT (JSON) ---
    {blueprint_json_string}
    --- END SEMANTIC BLUEPRINT ---

    Adhere strictly to the blueprint's instructions, style guides, and goals. The blueprint defines HOW you write; the source material defines WHAT you write about.
    """

    user_prompt = f"""
    --- SOURCE MATERIAL ({source_label}) ---
    {source_material}
    --- END SOURCE MATERIAL ---

    Generate the content now, following the blueprint precisely.
    """

    # Generate the final content (slightly higher temperature for potential creativity)
    final_output = call_llm_robust(system_prompt, user_prompt)

    # The output content IS the generated text (as a string)
    return create_mcp_message("Writer", final_output)

print("Specialist Agents defined.")

Specialist Agents defined.


In [11]:
#@title 5.The Agent Registry (The Toolkit)
# -------------------------------------------------------------------------
# We formalize the "Handler Registry" into an AgentRegistry.
# This catalogs agents and describes their capabilities to the Planner.
# -------------------------------------------------------------------------

class AgentRegistry:
    def __init__(self):
        # Mapping of agent names to their corresponding functions
        self.registry = {
            "Librarian": agent_context_librarian,
            "Researcher": agent_researcher,
            "Writer": agent_writer,
        }

    def get_handler(self, agent_name):
        """Retrieves the function associated with an agent name."""
        handler = self.registry.get(agent_name)
        if not handler:
            raise ValueError(f"Agent '{agent_name}' not found in registry.")
        return handler

    def get_capabilities_description(self):
        """
        Returns a structured description of the agents for the Planner LLM.
        This is crucial for the Planner to understand how to use the agents.
        """
        return """
        Các Tác nhân (Agents) hiện có và các đầu vào bắt buộc của chúng:

        1. TÁC NHÂN (AGENT): Librarian
           VAI TRÒ (ROLE): Truy xuất các Bản thiết kế Ngữ nghĩa (các hướng dẫn về phong cách/cấu trúc).
           ĐẦU VÀO (INPUTS):
             - "intent_query": (Chuỗi) Một cụm từ mô tả về phong cách hoặc định dạng mong muốn.
           ĐẦU RA (OUTPUT): Cấu trúc bản thiết kế (chuỗi JSON).

        2. TÁC NHÂN (AGENT): Researcher
           VAI TRÒ (ROLE): Truy xuất và tổng hợp thông tin thực tế về một chủ đề.
           ĐẦU VÀO (INPUTS):
             - "topic_query": (Chuỗi) Chủ đề cần nghiên cứu.
           ĐẦU RA (OUTPUT): Các dữ kiện đã được tổng hợp (Chuỗi).

        3. TÁC NHÂN (AGENT): Writer
           VAI TRÒ (ROLE): Tạo mới hoặc viết lại nội dung bằng cách áp dụng Bản thiết kế (Blueprint) vào tài liệu nguồn.
           ĐẦU VÀO (INPUTS):
             - "blueprint": (Chuỗi/Tham chiếu) Các hướng dẫn về phong cách (thường từ Librarian).
             - "facts": (Chuỗi/Tham chiếu) Thông tin thực tế (thường từ Researcher). Sử dụng thông tin này để tạo nội dung mới.
             - "previous_content": (Chuỗi/Tham chiếu) Văn bản hiện có (thường từ một bước Writer trước đó). Sử dụng thông tin này để viết lại hoặc điều chỉnh nội dung.
           ĐẦU RA (OUTPUT): Văn bản cuối cùng được tạo ra (Chuỗi).
        """

# Initialize the global toolkit
AGENT_TOOLKIT = AgentRegistry()
print("Agent Registry initialized.")

Agent Registry initialized.


In [ ]:
#@title 6.The Context Engine (Planner, Executor, Tracer)
# -------------------------------------------------------------------------
# This is the core innovation of Chapter 4. It replaces the linear
# Orchestrator with a dynamic, LLM-driven planning and execution system.
# -------------------------------------------------------------------------

# === 6.1. The Tracer (Debugging Implementation) ===
class ExecutionTrace:
    """Logs the entire execution flow for debugging and analysis."""
    def __init__(self, goal):
        self.goal = goal
        self.plan = None
        self.steps = []
        self.status = "Initialized"
        self.final_output = None
        self.start_time = time.time()

    def log_plan(self, plan):
        self.plan = plan

    def log_step(self, step_num, agent, planned_input, mcp_output, resolved_input):
        """Logs the details of a single execution step."""
        self.steps.append({
            "step": step_num,
            "agent": agent,
             # The raw input definitions from the plan (including $$REFS$$)
            "planned_input": planned_input,
            # Crucial for debugging: What exact context did the agent receive?
            "resolved_context": resolved_input,
            "output": mcp_output['content']
        })

    def finalize(self, status, final_output=None):
        self.status = status
        self.final_output = final_output
        self.duration = time.time() - self.start_time

    def display_trace(self):
        """Displays the trace in a readable format."""
        display(Markdown(f"### Execution Trace\n**Goal:** {self.goal}\n**Status:** {self.status} (Duration: {self.duration:.2f}s)"))
        if self.plan:
            # Display the raw plan JSON
            display(Markdown(f"#### Plan:\n```json\n{json.dumps(self.plan, indent=2)}\n```"))

        display(Markdown("#### Execution Steps:"))
        for step in self.steps:
            print(f"--- Step {step['step']}: {step['agent']} ---")
            print("  [Planned Input]:", step['planned_input'])
            # print("  [Resolved Context]:", textwrap.shorten(str(step['resolved_context']), width=150))
            print("  [Output Snippet]:", textwrap.shorten(str(step['output']), width=150))
            print("-" * 20)


# === 6.2. The Planner (Strategic Analysis) ===
def planner(goal, capabilities):
    """
    Analyzes the goal and generates a structured Execution Plan using the LLM.
    """
    print("[Engine: Planner] Analyzing goal and generating execution plan...")
    system_prompt = f"""
    Bạn là bộ não chiến lược (strategic core) của Context Engine. Nhiệm vụ của bạn là phân tích mục tiêu cấp cao của người dùng và thiết lập một Kế hoạch Thực thi (Execution Plan) có cấu trúc bằng cách phối hợp các agent sẵn có.

    --- NĂNG LỰC HIỆN CÓ (AVAILABLE CAPABILITIES) ---
    {capabilities}
    --- KẾT THÚC NĂNG LỰC ---

    HƯỚNG DẪN BẮT BUỘC:
    1. Định dạng phản hồi: Kế hoạch PHẢI là một danh sách JSON (JSON list) chứa các đối tượng, mỗi đối tượng đại diện cho một "step" (bước).
    2. Context Chaining: Bạn PHẢI sử dụng cơ chế chuỗi ngữ cảnh. Nếu một bước cần dữ liệu đầu vào từ một bước trước đó, hãy tham chiếu bằng cú pháp: $$STEP_X_OUTPUT$$.
    3. Tư duy Chiến lược: Chia nhỏ các mục tiêu phức tạp (ví dụ: quy trình viết lại tuần tự) thành các bước riêng biệt. 
    4. Độ chính xác của Input Keys: Sử dụng đúng các khóa đầu vào cho agent Writer: 
       - 'facts': Dùng cho dữ liệu thực tế/thông tin mới.
       - 'previous_content': Dùng cho nội dung cần được chỉnh sửa hoặc viết lại.

    MỤC TIÊU VÍ DỤ: "Viết một câu chuyện hồi hộp về tàu Apollo 11."
    KẾ HOẠCH VÍ DỤ (JSON LIST):
    [
        {{"step": 1, "agent": "Librarian", "input": {{"intent_query": "suspenseful narrative blueprint"}}}},
        {{"step": 2, "agent": "Researcher", "input": {{"topic_query": "Apollo 11 landing details"}}}},
        {{"step": 3, "agent": "Writer", "input": {{"blueprint": "$$STEP_1_OUTPUT$$", "facts": "$$STEP_2_OUTPUT$$"}}}}
    ]

    MỤC TIÊU VÍ DỤ: "Viết báo cáo kỹ thuật về tàu Juno, sau đó viết lại nội dung đó theo phong cách bình dân."
    KẾ HOẠCH VÍ DỤ (JSON LIST):
    [
        {{"step": 1, "agent": "Librarian", "input": {{"intent_query": "technical report structure"}}}},
        {{"step": 2, "agent": "Researcher", "input": {{"topic_query": "Juno mission technology"}}}},
        {{"step": 3, "agent": "Writer", "input": {{"blueprint": "$$STEP_1_OUTPUT$$", "facts": "$$STEP_2_OUTPUT$$"}}}},
        {{"step": 4, "agent": "Librarian", "input": {{"intent_query": "casual summary style"}}}},
        {{"step": 5, "agent": "Writer", "input": {{"blueprint": "$$STEP_4_OUTPUT$$", "previous_content": "$$STEP_3_OUTPUT$$"}}}}
    ]

    CHỈ phản hồi duy nhất danh sách JSON, không kèm theo lời giải thích nào khác.
    """
    # Call LLM in JSON mode for reliability
    plan_json = ""
    try:
        plan_json = call_llm_robust(system_prompt, goal, json_mode=True)
        plan = json.loads(plan_json)

        if not isinstance(plan, list):
             # Handle cases where the LLM wraps the list in a dictionary
             if isinstance(plan, dict):
                 if "plan" in plan and isinstance(plan["plan"], list):
                     plan = plan["plan"]
                 elif "steps" in plan and isinstance(plan["steps"], list): # <--- ADD THIS CHECK
                     plan = plan["steps"]
                 else:
                    raise ValueError("Planner returned a dict, but missing 'plan' or 'steps' key.")
             else:
                raise ValueError("Planner did not return a valid JSON list structure.")

        print("[Engine: Planner] Plan generated successfully.")
        return plan
    except Exception as e:
        print(f"[Engine: Planner] Failed to generate a valid plan. Error: {e}. Raw LLM Output: {plan_json}")
        raise e


# === 6.3. The Executor (Context Assembly and Execution) ===

def resolve_dependencies(input_params, state):
    """
    Helper function to replace $$REF$$ placeholders with actual data from the execution state.
    This implements Context Chaining.
    """
    # Use copy.deepcopy to ensure the original plan structure is not modified
    resolved_input = copy.deepcopy(input_params)

    # Recursive function to handle potential nested structures
    def resolve(value):
        if isinstance(value, str) and value.startswith("$$") and value.endswith("$$"):
            ref_key = value[2:-2]
            if ref_key in state:
                # Retrieve the actual data (string) from the previous step's output
                print(f"[Engine: Executor] Resolved dependency {ref_key}.")
                return state[ref_key]
            else:
                raise ValueError(f"Dependency Error: Reference {ref_key} not found in execution state.")
        elif isinstance(value, dict):
            return {k: resolve(v) for k, v in value.items()}
        elif isinstance(value, list):
            return [resolve(v) for v in value]
        return value

    return resolve(resolved_input)


def context_engine(goal):
    """
    The main entry point for the Context Engine. Manages Planning and Execution.
    """
    print(f"\n=== [Context Engine] Starting New Task ===\nGoal: {goal}\n")
    trace = ExecutionTrace(goal)
    registry = AGENT_TOOLKIT

    # Phase 1: Plan
    try:
        capabilities = registry.get_capabilities_description()
        plan = planner(goal, capabilities)
        trace.log_plan(plan)
    except Exception as e:
        trace.finalize("Failed during Planning")
        # Return the trace even in failure for debugging
        return None, trace

    # Phase 2: Execute
    # State stores the raw outputs (strings) of each step: { "STEP_X_OUTPUT": data_string }
    state = {}

    for step in plan:
        step_num = step.get("step")
        agent_name = step.get("agent")
        planned_input = step.get("input")

        print(f"\n[Engine: Executor] Starting Step {step_num}: {agent_name}")

        try:
            handler = registry.get_handler(agent_name)

            # Context Assembly: Resolve dependencies
            resolved_input = resolve_dependencies(planned_input, state)

            # Execute Agent via MCP
            # Create an MCP message with the RESOLVED input for the agent
            mcp_resolved_input = create_mcp_message("Engine", resolved_input)
            mcp_output = handler(mcp_resolved_input)

            # Update State and Log Trace
            output_data = mcp_output["content"]

            # Store the output data (the context itself)
            state[f"STEP_{step_num}_OUTPUT"] = output_data
            trace.log_step(step_num, agent_name, planned_input, mcp_output, resolved_input)
            print(f"[Engine: Executor] Step {step_num} completed.")

        except Exception as e:
            error_message = f"Execution failed at step {step_num} ({agent_name}): {e}"
            print(f"[Engine: Executor] ERROR: {error_message}")
            trace.finalize(f"Failed at Step {step_num}")
            # Return the trace for debugging the failure
            return None, trace

    # Finalization
    final_output = state.get(f"STEP_{len(plan)}_OUTPUT")
    trace.finalize("Success", final_output)
    print("\n=== [Context Engine] Task Complete ===")

    # Return the output of the final step AND the trace
    return final_output, trace